In [6]:
from omegaconf import DictConfig, OmegaConf
import inspect

import shutil

from pathlib import Path
from optbench import functions
from optbench.abstract_function import AbstractFunction

def fullname(o):
    try:
        # if o is a class or function, get module directly
        module = o.__module__
    except AttributeError:
        # then get module from o's class
        module = o.__class__.__module__
    try:
        # if o is a class or function, get name directly
        name = o.__qualname__
    except AttributeError:
        # then get o's class name
        name = o.__class__.__qualname__
    # if o is a method of builtin class, then module will be None
    if module == 'builtins' or module is None:
        return name
    return module + '.' + name

base_path = Path("../optbench/configs/problem")

# Problem ids
benchmark_id = "OptBench"

target_path = base_path / benchmark_id
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "optbench.optbench_problem.OptBenchProblem"

function_classes = [
    c[1]
    for c in inspect.getmembers(functions, inspect.isclass)
    if issubclass(c[1], AbstractFunction) and c[1] != AbstractFunction
]
dimensions = [2, 5, 10]


def get_n_trials(dim: int) -> int:
    return dim * 25


for function_class in function_classes:
    for dim in dimensions:
        problem_id = f"{function_class.__name__}-{dim}"
        cfg = DictConfig(
            {
                "benchmark_id": benchmark_id,
                "problem_id": problem_id,
                "problem": {
                    "_target_": problem_class,
                    "function": {
                        "_target_": fullname(function_class),
                        "dim": dim,
                        "seed": "${seed}",
                    },
                },
                "task": {
                    "n_trials": get_n_trials(dim),
                    "n_objectives": 1,
                },
            }
        )
        fn = target_path / f"{function_class.__name__}_{dim}.yaml"
        yaml_str = OmegaConf.to_yaml(cfg=cfg)
        yaml_str = "# @package _global_\n" + yaml_str
        fn.write_text(yaml_str)
        print(cfg)

{'benchmark_id': 'OptBench', 'problem_id': 'Ackley-2', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Ackley', 'dim': 2, 'seed': '${seed}'}}, 'task': {'n_trials': 50, 'n_objectives': 1}}
{'benchmark_id': 'OptBench', 'problem_id': 'Ackley-5', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Ackley', 'dim': 5, 'seed': '${seed}'}}, 'task': {'n_trials': 125, 'n_objectives': 1}}
{'benchmark_id': 'OptBench', 'problem_id': 'Ackley-10', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Ackley', 'dim': 10, 'seed': '${seed}'}}, 'task': {'n_trials': 250, 'n_objectives': 1}}
{'benchmark_id': 'OptBench', 'problem_id': 'Levy-2', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Levy', 'dim': 2, 'seed': '${seed}'}}, 'task': {'n_trials': 50, 'n_o